## Описание проекта: Прогнозирование оттока клиентов для оператора "ТелеДом"

### Цель проекта
Оператор связи «ТелеДом» стремится снизить отток клиентов. Для этого необходимо:
- Выявлять абонентов, планирующих отказаться от услуг
- Предлагать им персональные промокоды и специальные условия
- Создать модель прогнозирования оттока клиентов

### Постановка задачи
Необходимо разработать модель машинного обучения, которая на основе исторических данных будет предсказывать вероятность расторжения договора клиентом.

### Данные
Для обучения модели доступны:
1. Персональные данные клиентов (анонимизированные)
2. Информация о подключенных тарифах
3. Данные об используемых услугах
4. Исторические данные о расторжении договоров

In [ ]:
!pip install -qq sweetviz
!pip install -qq optuna
!pip install -qq phik
!pip install -qq optuna-integration[sklearn]
!pip install -qq scikit-learn

In [ ]:
import pandas as pd
import sweetviz as sv
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import optuna
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from lightgbm import LGBMClassifier, plot_importance
from phik import phik_matrix
from sqlalchemy import create_engine
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, PowerTransformer, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_report, precision_recall_curve, average_precision_score
from optuna.integration import OptunaSearchCV
from optuna.samplers import TPESampler
from optuna.distributions import IntDistribution, FloatDistribution, CategoricalDistribution
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
RANDOM_STATE = 300625
TEST_SIZE = 0.25
PATH_TO_DB = "db/ds-plus-final.db"
torch.manual_seed(RANDOM_STATE)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

Импорт библиотек и настройка окружения

P.S. Путь к файлу базы данных находится в константе PATH_TO_DB

## Загрузка данных

In [ ]:
ENGINE = create_engine(f"sqlite:///{PATH_TO_DB}", echo=False)

In [ ]:
sql_query = '''
    SELECT 
        name 
    FROM 
        sqlite_master
    WHERE 
        type='table';
'''
tables_name = pd.read_sql_query(sql_query, con=ENGINE)
tables_name

Проверка на существование таблиц в схеме, помимо нужных нам четырех таблиц есть другие, использовать их не будем

In [ ]:
def get_data(query):
    try:
        return pd.read_sql_query(query, con=ENGINE)
    except Exception as e:
        print(f'Error: {e}')
        return None

In [ ]:
contract_query = """
    SELECT 
        c.*
    FROM 
        contract c;
"""
contract = get_data(contract_query)

In [ ]:
personal_query = """
    SELECT 
        p.*
    FROM 
        personal p;
"""
personal = get_data(personal_query)

In [ ]:
internet_query = """
    SELECT 
        i.*
    FROM 
        internet i;
"""
internet = get_data(internet_query)

In [ ]:
phone_query = """
    SELECT 
        p.*
    FROM 
        phone p;
"""
phone = get_data(phone_query)

## Исследовательский анализ и предобработка данных

In [ ]:
df_dict = {
    "contract": contract,
    "personal": personal,
    "internet": internet,
    "phone": phone
}

for name, df in df_dict.items():
    print(f"\n=== Анализ таблицы {name} ===\n")
    
    print("Первые 5 строк:")
    display(df.head())
    print("\nИнформация о структуре:")
    print(df.info())
    
    print("\nПропущенные значения:")
    print(df.isna().sum())

### Промежуточные выводы:

**Качество данных**:
- Во всех таблицах нет пропусков
- Все данные в таблицах имеют тип данных object, в будущем нужно преобразовать в нужный для нас

**Охват услуг**:
- Всего клиентов: 7,043
- Интернет: 5,517 (78%)
- Телефония: 6,361 (90%) 

### Предобарботка

In [ ]:
contract['target'] = 1
contract.loc[contract['EndDate'] == 'No', 'target'] = 0

In [ ]:
contract['TotalCharges'] = contract['TotalCharges'].replace(' ', '0')

In [ ]:
contract.BeginDate = pd.to_datetime(contract.BeginDate)
contract["EndDate"] = contract['EndDate'].replace('No', "2020-02-01")
contract["EndDate"] = pd.to_datetime(contract["EndDate"])
days_diff = contract["EndDate"] - contract["BeginDate"]
contract["duration_contract"] = days_diff.dt.components.days

In [ ]:
contract.MonthlyCharges = contract.MonthlyCharges.astype("float")
contract.TotalCharges = contract.TotalCharges.astype("float")
phone = phone.rename(columns={"CustomerId":"customerID"})

Создана **Целевая переменная (target)**:
- `target = 1` если клиент ушел (есть EndDate)
- `target = 0` если активный клиент (EndDate = 'No')

**Работа с датами**:
- Преобразованы в datetime: `BeginDate` и `EndDate`
- Добавлен новый признак `duration_contract` (длительность договора в днях)

**Числовые поля**:
- `MonthlyCharges` и `TotalCharges` преобразованы в float
- Удалены строки с пустыми значениями в `TotalCharges` (пробелы)

**Остальное**:
- В таблице `personal` преобразован `SeniorCitizen` в int
- В таблице `phone` переименован столбец `CustomerId` → `customerID` для будущего объединения по столбцу

### Исследовательский анализ

In [ ]:
report_contract = sv.analyze(contract)
report_contract.show_notebook()

#### Вывод по анализу таблицы contract

##### Общая информация
- **Записей**: 7,032 (без дубликатов)
- **Признаки**: 10 (6 категориальных, 3 числовых, 1 текстовый)

##### Распределение тарифов
- **Month-to-month**: 55% 
- **Двухлетние**: 24%  
- **Годовые**: 21%

##### Целевая переменная (target)
- **Активные клиенты (0)**: 84% (5,931)
- **Ушедшие (1)**: 16% (1,101)

Из числовых признаков нормальное распределение имеет только столбец **MonthlyCharges** с ежемесячными тратами, также у него есть выбросы, все это учтем на стадии преобразования данных

In [ ]:
report_personal = sv.analyze(personal)
report_personal.show_notebook()

#### Вывод по анализу таблицы personal

##### Общая информация
- **Записей**: 7,043 (уникальные customerID)
- **Признаки**: 5 демографических характеристик

##### Гендерный состав
- **Мужчины**: 50% (3,555)
- **Женщины**: 50% (3,488)

##### Возрастные группы
- **Пенсионеры**: 16% (1,142)
- **Не пенсионеры**: 84% 

##### Семейное положение
- **С партнером**: 48% (3,402)
- **Без партнера**: 52%

##### Наличие иждивенцев
- **С иждивенцами**: 30% (2,110)
- **Без иждивенцев**: 70%


In [ ]:
report_internet = sv.analyze(internet)
report_internet.show_notebook()

#### Вывод по анализу таблицы internet

##### Общая информация
- **Записей**: 5,517 (78% от общего числа клиентов)
- **Признаки**: 7 категориальных (услуги интернета)

##### Типы подключения
- **Fiber optic**: 56% (3,096)
- **DSL**: 44% (2,421)

##### Дополнительные услуги
| Услуга               | Доля "No" | Доля "Yes" |
|----------------------|----------|----------|
| OnlineSecurity       | 63%      | 37%      |
| TechSupport          | 63%      | 37%      |
| OnlineBackup         | 56%      | 44%      |
| DeviceProtection     | 56%      | 44%      |
| StreamingTV         | 51%      | 49%      |
| StreamingMovies     | 50%      | 50%      |

In [ ]:
report_phone = sv.analyze(phone)
report_phone.show_notebook()

#### Вывод по анализу таблицы phone

##### Общая информация
- **Записей**: 6,361 (90% от общего числа клиентов)  
- **Признаки**: 1 категориальный 

##### Распределение услуги MultipleLines(подключение телефона к нескольким линиям одновременно)
- **No**: 53% (3,390 клиентов)  
- **Yes**: 47% (2,971 клиент)  


### Объединение таблиц по ID клиента

In [ ]:
data = contract.merge(personal, on='customerID', how='left')
data = data.merge(internet, on='customerID', how='left')
data = data.merge(phone, on='customerID', how='left')
service_columns = [
    'InternetService',
    'OnlineSecurity',
    'OnlineBackup',
    'DeviceProtection',
    'TechSupport',
    'StreamingTV',
    'StreamingMovies',
    'MultipleLines'
]

for col in service_columns:
    data[col] = data[col].fillna('No service')

In [ ]:
data["year"] = data.BeginDate.dt.year
data["month"] = data.BeginDate.dt.month
data["day"] = data.BeginDate.dt.day
service_cols = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
               'TechSupport', 'StreamingTV', 'StreamingMovies']

data['additional_services'] = data[service_cols].apply(
    lambda row: 'Yes' if row.isin(['Yes']).any() else 'No', axis=1)
current_date = pd.to_datetime('2020-02-01')

In [ ]:
object_cols = ['Type', 'PaperlessBilling', 'PaymentMethod', 'gender',
               'Partner', 'Dependents', 'InternetService', 'OnlineSecurity',
               'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
               'StreamingMovies', 'MultipleLines', 'additional_services', 'SeniorCitizen']

for col in object_cols:
    if col in data.columns:
        data[col] = data[col].astype('category')

In [ ]:
data.head()

Таблицы объединены в одну - data, данные приведены к нужным типам и созданые новые признаки

In [ ]:
plt.figure(figsize=(15, 13))
sns.heatmap(phik_matrix(data[[
    'target',
    'TotalCharges',
    'SeniorCitizen',
    'Type',
    'PaperlessBilling',
    'PaymentMethod',
    'Partner',
    'Dependents',
    'MultipleLines',
    'duration_contract',
    'MonthlyCharges',
]], interval_cols=['MonthlyCharges', 'duration_contract', 'TotalCharges']), annot=True, cmap='coolwarm')
plt.title('Матрица корреляции признаков')
plt.show()

Для обучения будут использовать следующие признаки:
- 'MonthlyCharges',
- 'duration_contract',
- 'TotalCharges'
- 'Type',    
- 'SeniorCitizen',
- 'PaperlessBilling', 
- 'PaymentMethod',
- 'Partner',    
- 'Dependents',
- 'MultipleLines',
- 'InternetService'

Между собой эти признаки не имеют мультиколлинеарности и по разному влияют на решение модели

In [ ]:
num_cols = [
    'MonthlyCharges',
    'duration_contract',
    'TotalCharges'             
]

cat_cols = [
    'Type',    
    'SeniorCitizen',
    'PaperlessBilling', 
    'PaymentMethod',
    'Partner',    
    'Dependents',
    'MultipleLines',
    'InternetService',
]

all_cols = num_cols + cat_cols

In [ ]:
print(data[num_cols].describe())
data[num_cols].hist(bins=30, figsize=(15, 5))
plt.show()

###  Основные выводы EDA

####  Числовые признаки

| Признак             | Описание                                                                 |
|---------------------|--------------------------------------------------------------------------|
| `MonthlyCharges`    | Диапазон: \$18.25–\$118.75<br>Среднее: \$65 (медиана \$70.35)           |
| `duration_contract` | Среднее: 900 дней (2.5 года)<br>Минимум: 28 дней                         |
| `TotalCharges`      | Макс: \$9221                    |

In [ ]:
for col in cat_cols:
    print(f"\n--- {col} ---")
    print(data[col].value_counts(normalize=True))
    sns.countplot(x=col, hue='target', data=data)
    plt.xticks(rotation=45)
    plt.show()


### Категориальные признаки

1. 55% клиентов на помесячной оплате (`Month-to-month`)
2. 44% используют оптоволокно (`Fiber optic`)
3. 34% платят электронными чеками 

### Подготовка данных для обучения

In [ ]:
data.info()

In [ ]:
X = data[all_cols]
y = data.target

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    shuffle=True,
    stratify=y)

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
def data_preparation_ohe(
    numeric_cols=num_cols,
    category_cols=cat_cols,
    X_train=X_train,
    X_test=X_test,
    power_transform_method='yeo-johnson'):
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', Pipeline([
                ('scaler', RobustScaler()), 
                ('power', PowerTransformer(method=power_transform_method))
        ]), numeric_cols),
            ('cat', OneHotEncoder(
                handle_unknown='ignore', 
                sparse_output=False,
                drop='first'
            ), category_cols)
        ], remainder='passthrough')

    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)

    feature_names = preprocessor.get_feature_names_out()
    X_train_df = pd.DataFrame(X_train_processed, columns=feature_names)
    X_test_df = pd.DataFrame(X_test_processed, columns=feature_names)
    
    return X_train_df, X_test_df

X_train_ohe, X_test_ohe = data_preparation_ohe()

In [ ]:
X_train_ohe.shape, X_test_ohe.shape

Данные подготовлены для обучения моделей, разделены на выборки, в каждой в каждой выборке есть метки каждого класса

## Обучение моделей

Рассмотрим следующие классы моделей:
- **случайный лес**
- **бустинг(LightGBM)**
- **нейронную сеть** на pytorch

### Обучение модели **LightGBM**

In [ ]:
params = {
    'learning_rate': FloatDistribution(0.0001, 0.2, log=True),
    'num_leaves': IntDistribution(100, 400),
    'max_depth': IntDistribution(3, 15),
    'min_child_samples': IntDistribution(10, 200),
    'subsample': FloatDistribution(0.5, 1.0),
    'colsample_bytree': FloatDistribution(0.5, 1.0),
    'reg_alpha': FloatDistribution(1e-5, 10.0, log=True),
    'reg_lambda': FloatDistribution(1e-5, 10.0, log=True)
}

model_lgb = LGBMClassifier(objective='binary', 
                           random_state=RANDOM_STATE,
                           n_estimators=2000,
                           verbose=100,
                           verbosity=-1)

In [ ]:
optuna_search_lgb = OptunaSearchCV(
    model_lgb,
    params,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE),
    scoring='roc_auc',
    n_trials=500,
    n_jobs=-1,
    verbose=1,
    random_state=RANDOM_STATE
)

optuna_search_lgb.fit(X_train, y_train)

print("Best params:", optuna_search_lgb.best_params_)
print("Best AUC:", optuna_search_lgb.best_score_)

### Вывод по обучению LightGBM

#### Результаты оптимизации:
- **Метрика AUC-ROC**: 0.910 
- **Время оптимизации**: ~8 минут (500 trials)

### Обучение модели **RandomForest**

In [ ]:
params_rf = {
    'n_estimators': IntDistribution(100, 1000),
    'max_depth': IntDistribution(3, 20),
    'min_samples_split': IntDistribution(2, 20),
    'min_samples_leaf': IntDistribution(1, 20),
    'max_features': CategoricalDistribution(['sqrt', 'log2', None]),
    'bootstrap': CategoricalDistribution([True, False]),
    'class_weight': CategoricalDistribution(['balanced', 'balanced_subsample', None]),
    'ccp_alpha': FloatDistribution(0.0, 0.1),
}

model_rf = RandomForestClassifier(random_state=RANDOM_STATE)
optuna_search_rf = OptunaSearchCV(
    model_rf,
    params_rf,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE),
    scoring='roc_auc',
    n_trials=30,
    n_jobs=-1,
    verbose=1,
    random_state=RANDOM_STATE
)

optuna_search_rf.fit(X_train_ohe, y_train)

print("\nBest params:", optuna_search_rf.best_params_)
print("Best AUC (CV):", optuna_search_rf.best_score_)

### Вывод по обучению RandomForest

#### Результаты оптимизации:
- **Метрика AUC-ROC**: 0.820
- **Время оптимизации**: ~2 минута (30 trials)

### Обучение нейронной сети

In [ ]:
class TelecomNeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size=4096, dropout_rate=0.2):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.LeakyReLU(),
            nn.Dropout(dropout_rate),
            
            nn.Linear(hidden_size, hidden_size//2),
            nn.BatchNorm1d(hidden_size//2),
            nn.LeakyReLU(),
            nn.Dropout(dropout_rate),
            
            nn.Linear(hidden_size//2, hidden_size//4),
            nn.BatchNorm1d(hidden_size//4),
            nn.LeakyReLU(),
            nn.Dropout(dropout_rate),

            nn.Linear(hidden_size//4, hidden_size//8),
            nn.BatchNorm1d(hidden_size//8),
            nn.LeakyReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size//8, 1),
        )
    
    def forward(self, x):
        return self.layers(x)

def train_telecom_model(X_train, y_train, X_val=None, y_val=None, 
                       params=None, device='cpu', n_epochs=150, 
                       patience=15, batch_size=1024):
    
    # Convert data to tensors
    X_train_tensor = torch.FloatTensor(X_train.values).to(device)
    y_train_tensor = torch.FloatTensor(y_train.values).to(device)
    
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    X_val_tensor = torch.FloatTensor(X_val.values).to(device)
    
    model = TelecomNeuralNet(X_train.shape[1], 
                           hidden_size=params['hidden_size'],
                           dropout_rate=params['dropout_rate']).to(device)
    
    optimizer = optim.AdamW(model.parameters(), lr=params['lr'], weight_decay=1e-5)
    criterion = nn.BCEWithLogitsLoss()
    
    best_auc = 0
    best_weights = None
    no_improve = 0
    
    for epoch in range(n_epochs):
        model.train()
        epoch_loss = 0
        
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1))
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_tensor)
            val_probs = torch.sigmoid(val_outputs).cpu().numpy()
            val_auc = roc_auc_score(y_val, val_probs)
            
            if val_auc > best_auc:
                best_auc = val_auc
                best_weights = model.state_dict()
                no_improve = 0
            else:
                no_improve += 1
                
            if no_improve >= patience:
                print(f"Early stopping at epoch {epoch}")
                model.load_state_dict(best_weights)
                break
            
        print(f"Epoch {epoch:3d} | Loss: {epoch_loss/len(train_loader):.4f} | "
                f"Val AUC: {val_auc:.4f}")
    
    return model

In [ ]:
def objective(trial, X_train, y_train, device, n_splits=3):
    params = {
        'hidden_size': trial.suggest_categorical('hidden_size', [256, 512, 1024, 2048]),
        'dropout_rate': trial.suggest_float('dropout_rate', 0.1, 0.5),
        'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True),
        'batch_size': trial.suggest_categorical('batch_size', [128, 256, 512, 1024])
    }
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)
    cv_auc_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
        X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        model = train_telecom_model(
            X_train_fold, y_train_fold, X_val_fold, y_val_fold,
            params=params, device=device, n_epochs=2000, patience=500,
            batch_size=params['batch_size']
        )
        
        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.FloatTensor(X_val_fold.values).to(device)
            val_outputs = model(X_val_tensor)
            val_probs = torch.sigmoid(val_outputs).cpu().numpy()
            val_auc = roc_auc_score(y_val_fold, val_probs)
            cv_auc_scores.append(val_auc)
        
        print(f"Fold {fold+1} AUC: {val_auc:.4f}")
    
    mean_auc = np.mean(cv_auc_scores)
    trial.set_user_attr("cv_auc_scores", cv_auc_scores)
    
    return mean_auc

In [ ]:
study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=RANDOM_STATE))
study.optimize(lambda trial: objective(trial, X_train_ohe, y_train, device), 
              n_trials=1, n_jobs=-1)

best_params = study.best_params
print("Best params:", best_params)
print("Best AUC:", study.best_value)

### Вывод по обучению нейронной сети

#### Результаты оптимизации:
- **Метрика AUC-ROC**: 0.718
- **Время оптимизации**: ~2 минуты (1 trial)

Лучшей моделью оказалась **LightGBM** с итоговой метрикой ROC-AUC на кросс-валидации - 0.851

## Тестирование лучшей модели

In [ ]:
test_preds_proba_lgb = optuna_search_lgb.predict_proba(X_test)[:, 1]
test_preds_class_lgb = optuna_search_lgb.predict(X_test)  

test_auc = roc_auc_score(y_test, test_preds_proba_lgb)
test_accuracy = accuracy_score(y_test, test_preds_class_lgb)
print(f"Test AUC: {test_auc:.4f}")
print(f"Accuracy: {test_accuracy:.4f}")

На тестовых данных метрика ROC-AUC составляет **0.9297**, что удовлетворяет условиям задачи, точность модели составляет **0.9319**, что является хорошим результатом

### Интерпретация результатов

#### ROC-AUC = 0.9297
 **Отличное качество разделения классов**   
 Вероятность корректного ранжирования: **92.97%**  
 Существенно превышает минимальное требование (0.85)

#### Accuracy = 0.9319
 **Общая точность предсказаний**: 93.19% значит, что модель предсказывает правильно с вероятность чуть выше 93%


In [ ]:
best_model = optuna_search_lgb.best_estimator_  

plt.figure(figsize=(10, 8))
plot_importance(best_model, max_num_features=20, importance_type='gain')
plt.title('Важность признаков модели LightGBM')
plt.show()

Самым важным признаком для модели является длительность контракта с клиентом

In [ ]:
data['duration_bins'] = pd.cut(data['duration_contract'],
                              bins=[0, 30, 90, 180, 365, 730, 1095, 1460, 1825],
                              labels=['<1 мес', '1-3 мес', '3-6 мес', '6-12 мес',
                                     '1-2 года', '2-3 года', '3-4 года', '>4 лет'])

plt.figure(figsize=(12, 6))
sns.barplot(x='duration_bins', y='target', data=data,
           palette='viridis', estimator=np.mean)
plt.title('Средний процент оттока по длительности контракта')
plt.xlabel('Длительность контракта')
plt.ylabel('Доля ушедших клиентов')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()

Из графика видно, что больше всего людей уходят в первый месяц после заключения договора об оказании услуги, стоит обратить на это большое внимание и придумать способы удержания клиента хотя бы на 3 месяца

In [ ]:
cm = confusion_matrix(y_test, test_preds_class_lgb)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Предсказание')
plt.ylabel('Факт')
plt.title('Матрица ошибок')
plt.show()

print(classification_report(y_test, test_preds_class_lgb))

####  Бизнес-интерпретация

##### True Negative (1471):
 **Лояльные клиенты правильно идентифицированы**  
 *Бизнес-эффект*:  
- Экономия бюджета на промо-акциях для 1464 клиентов  
- Не создаем лишний контакт с довольными клиентами

##### False Positive (15):
 **Лояльные клиенты ошибочно помечены как уходящие**  
 *Бизнес-риск*:  
- 19 клиентов получат ненужные промо-предложения  

##### False Negative (105):
 **Уходящие клиенты не обнаружены**  
 *Бизнес-потери*:  
- Потенциальная потеря 109 клиентов 

##### True Positive (170):
 **Уходящие клиенты правильно выявлены**  
 *Бизнес-возможность*:  
- 166 клиентов могут быть удержаны специальными предложениями

In [ ]:
y_scores = best_model.predict_proba(X_test)[:, 1]

precision, recall, _ = precision_recall_curve(y_test, y_scores)
ap_score = average_precision_score(y_test, y_scores)

plt.figure()
plt.plot(recall, precision, label=f'AP = {ap_score:.2f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Кривая Precision-Recall')
plt.legend()
plt.show()

В ходе выполнения проекта - Прогнозирование оттока клиентов для оператора "ТелеДом" были выполнены следующие шаги :

- Выгрузка и анализ таблиц;
- Предобработка данных;
- Объединение в один датасет для дальнейшего обучения моделей;
- Выбор и анализ признаков для обучения;
- Подготовка данных к обучению - кодировка и разделение на выборки;

Всего обучили три модели: RandomForestClassifier, LightGBM, NeuralNetwork. 
По итогу обучения выбрали модель градиентного бустинга LightGBM, она легкая не требует больших вычислительных ресурсов, имеет быстрое время обучения и предсказания, также имеет высокую метрику на тестовых данных ROC-AUC - 0.9186

После обучения модели на стадии определения важных признаков при обучении было выяснено, что модель очень большое внимание уделяет длительности договора с клиентом и оказалось, что огромное количество людей отказываются от услуг в первый же месяц после заключения договора, рекомендуется ввести различные акции для новых клиентов, которые длятся, хотя бы 3 месяца, акции и услуги должны быть максимально привлекательными и выгодными для клиента.